In [1]:
import numpy as np
import os
os.chdir("C:\\Users\\ma11115\\OneDrive - Imperial College London\\github_folder\\STK_SEARCH")

In [2]:
#%pip install pytorch-lightning

import os
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl


In [3]:
import numpy as np
slatm_rpr = np.load('data/output/Full_dataset/repr_df_total_new2023_08_20.npy',allow_pickle=True)
slatm_name = np.load('data/output/Full_dataset/names_df_total_new2023_08_20.npy',allow_pickle=True)

In [4]:
import pandas as pd
from src.stk_search import Database_utils
import importlib
importlib.reload(Database_utils)
#%% 
# Load the searched space
df_path = 'data/output/Full_datatset/df_total_new2023_08_20.csv'
df_precursors_path = 'data/output/Prescursor_data/calculation_data_precursor_190923_clean.pkl'#'Data/output/Prescursor_data/calculation_data_precursor_310823_clean.pkl'
df_total, df_precursors = Database_utils.load_data_from_file(df_path, df_precursors_path)


C:\Users\ma11115\OneDrive - Imperial College London\github_folder\STK_SEARCH\src\stk_search\Database_utils.py:106: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_total = pd.read_csv(df_path)


In [5]:
import numpy as np
slatm_rpr_precursor = np.load('data/output/Prescursor_data/repr_slatm_precursor_310823_clean.pkl.npy',allow_pickle=True)
slatm_name_precursor = np.load('data/output/Prescursor_data/names_slatm_precursor_310823_clean.pkl.npy',allow_pickle=True)
slatm_rpr_precursor = slatm_rpr_precursor[:, ~(slatm_rpr_precursor == slatm_rpr_precursor[0,:]).all(0)]
for i,x in enumerate(slatm_name_precursor):
    x=x.split('/')[1].replace('.xyz','')
    slatm_name_precursor[i]=x

In [6]:
#np.isnan(slatm_rpr).any()
import pandas as pd
slatm_rpr_new=slatm_rpr
print(slatm_rpr_new.shape)
slatm_rpr_new = slatm_rpr_new[:, ~(slatm_rpr_new == slatm_rpr_new[0,:]).all(0)]
print(slatm_rpr_new.shape)

(40975, 40221)
(40975, 14719)


In [7]:
for i in range(6):
    df_total[f'slatm_{i}']=df_total[f'InChIKey_{i}'].apply(lambda x: slatm_rpr_precursor[slatm_name_precursor==x][0])
init_slatm_rpr = np.stack(df_total[f'slatm_{0}'].values)
for i in range(1,6):
    init_slatm_rpr = np.concatenate((init_slatm_rpr,np.stack(df_total[f'slatm_{i}'].values)), axis=1)
init_slatm_rpr.shape

(40975, 25638)

In [8]:
print(slatm_rpr_new.shape)
print(init_slatm_rpr.shape)

(40975, 14719)
(40975, 25638)


In [9]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(25638, 100), nn.ReLU(), nn.Linear(100, 14719))

    def forward(self, x):
        return self.l1(x)

In [14]:
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        loss = F.mse_loss(z, y)
        self.log('train_loss', loss)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def validation_step(self, batch, batch_idx):
        '''used for logging metrics'''
        preds, loss, acc = self._get_preds_loss_accuracy(batch)

        # Log loss and metric
        self.log('val_loss', loss)
        self.log('val_accuracy', acc)
        return preds
    
    def _get_preds_loss_accuracy(self, batch):
        '''convenience function since train/valid/test steps are similar'''
        x, y = batch
        logits = self(x)
        preds = torch.argmax(logits, dim=1)
        loss = self.loss(logits, y)
        acc = accuracy(preds, y)
        return preds, loss, acc

In [34]:
from torch.utils.data import Dataset, DataLoader

# convert into PyTorch tensors
X = torch.tensor(init_slatm_rpr, dtype=torch.float32)
y = torch.tensor(slatm_rpr_new, dtype=torch.float32)

# create DataLoader, then take one batch
dataset = list(zip(X,y))
loader = DataLoader(dataset, shuffle=True, batch_size=160,num_workers=6)

In [45]:
dataset = list(zip(X,y))
train_len = int(len(dataset)*0.8)
val_len = len(dataset)-train_len

train_set, val_set = torch.utils.data.random_split(dataset, [train_len, val_len])
batch_size = 1000

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           num_workers=6,shuffle = True)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,shuffle = False)

In [46]:

# model
from pytorch_lightning.loggers import WandbLogger

import wandb
wandb.login()



autoencoder = LitAutoEncoder(Encoder())
wandb_logger = WandbLogger(log_model="all")
# train model
trainer = pl.Trainer(logger=wandb_logger,fast_dev_run=False, max_epochs=2,val_check_interval=1.0)
trainer.fit(model=autoencoder, train_dataloaders=train_loader, val_dataloaders=validation_loader)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..

  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 4.1 M 
------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.202    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

AttributeError: 'LitAutoEncoder' object has no attribute '_get_preds_loss_accuracy'

In [33]:
pred = autoencoder.encoder(X)
F.mse_loss(pred, y)

tensor(0.5623, grad_fn=<MseLossBackward0>)

tensor([ 0.,  4., 60.,  ...,  0.,  0.,  0.])